In [1]:
pip install python-terrier

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 102 kB 4.4 MB/s 
     |████████████████████████████████| 1.5 MB 8.3 MB/s 
     |████████████████████████████████| 69 kB 3.2 MB/s 
     |████████████████████████████████| 311 kB 6.7 MB/s 
     |████████████████████████████████| 46 kB 654 kB/s 
     |████████████████████████████████| 286 kB 22.7 MB/s 
     |████████████████████████████████| 45 kB 978 kB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 293 kB 24.7 MB/s 
     |████████████████████████████████| 72 kB 950 kB/s 
     |████████████████████████████████| 133 kB 23.4 MB/s 
     |████████████████████████████████| 1.2 MB 34.5 MB/s 
  Created wheel for python-terrier: filename=python_terrier-0.9.1-py3-none-any.whl size=110765 sha256=4015d28ad2350c71b7503cdb275862d916617213b66230

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 150)

import pyterrier as pt
import os

In [2]:
import platform
platform.uname()

uname_result(system='Windows', node='DESKTOP-O6IU41N', release='10', version='10.0.19045', machine='AMD64')

In [3]:
if not pt.started():
    pt.init()

PyTerrier 0.8.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30)



## Preprocessing

In [10]:
mpst = pd.read_csv("/content/drive/MyDrive/650data/mpst_full_data.csv")

In [11]:
mpst['docno'] = mpst.index + 1

In [12]:
mpst['plot_synopsis'] = mpst.plot_synopsis.astype(str)
mpst['docno'] = mpst.docno.astype(str)
mpst['title'] = mpst.title.astype(str)

In [13]:
review = pd.read_csv("/content/drive/MyDrive/650data/review.csv").iloc[:,1:]
review = review.rename(columns = {'imdbid':'imdb_id'}).dropna()
review['docno'] = review.index + 1
review['docno'] = review.docno.astype(str)

In [14]:
document = review.merge(mpst, on = ["imdb_id"], how = "inner").rename(columns = {"title_x":"title"})
final_document = document.drop(["title_y", 'docno_x', 'docno_y'], axis = 1)


In [15]:
final_document['title'] = final_document.title.astype(str)
final_document['imdb_id'] = final_document.imdb_id.astype(str)
final_document['review_title_1'] = final_document.review_title_1.astype(str) 
final_document['review_content_1'] = final_document.review_content_1.astype(str)
final_document['review_title_2'] = final_document.review_title_2.astype(str)
final_document['review_content_2'] = final_document.review_content_2.astype(str)
final_document['review_title_3'] = final_document.review_title_3.astype(str)
final_document['review_content_3'] = final_document.review_content_3.astype(str)
final_document['review_title_4'] = final_document.review_title_4.astype(str)
final_document['review_content_4'] = final_document.review_content_4.astype(str)
final_document['review_title_5'] = final_document.review_title_5.astype(str)
final_document['review_content_5'] = final_document.review_content_5.astype(str)
final_document[ 'plot_synopsis'] = final_document.plot_synopsis.astype(str)


In [16]:
final_document['docno'] = final_document.index + 1
final_document['docno'] = final_document.docno.astype(str)

## Indexing

In [18]:
index_dir = '/content/drive/MyDrive/650data/doc_index'
indexer = pt.DFIndexer(index_dir, overwrite=True, blocks = True)
index_ref = indexer.index(final_document['plot_synopsis'],final_document['docno'], final_document['review_title_1'], 
                          final_document['review_content_1'],final_document['review_title_2'], 
                          final_document['review_content_2'], final_document['review_title_3'],
                          final_document['review_content_3'], final_document['review_title_4'], 
                          final_document['review_content_4'],final_document['review_title_5'], 
                          final_document['review_content_5']
                          )
index_ref.toString()
index = pt.IndexFactory.of(index_ref)

In [19]:
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
DFIZ = pt.BatchRetrieve(index, wmodel="DFIZ")
DirichletLM = pt.BatchRetrieve(index, wmodel="DirichletLM")



In [20]:
query = pd.read_csv("/content/drive/MyDrive/650data/query.csv",encoding="latin1")
query['qid'] = query.qid.astype(str)
query['query'][16] = 'The first montage in the history of cinema directed by a Soviet director'
query#.head(20)

,qid,query
0,1,Spielbergs films that are suitable for family to watch
1,2,Oscar winning war films
2,3,Films that reflect Russian history
3,4,History films that have big budgets
4,5,Films depicting the modern history of China
5,6,A romantic comedy starring Zooey Deschanel
6,7,A small budget independent film directed by Wes Anderson
7,8,After entering Hollywood Jackie Chan starred in the action comedy film
8,9,A science fiction blockbuster reflecting the future war between humans and robots
9,10,Musical film adaptations of world-famous books


In [21]:
qrels = pd.read_csv("/content/drive/MyDrive/650data/annotation_done.csv")[['qid','docno','relevance_score']]
qrels['qid'] = qrels.qid.astype(str)
qrels['docno'] = qrels.docno.astype(str)
qrels = qrels.rename(columns = {'relevance_score':'relevance'})
qrels

,qid,docno,relevance
0,1,2234,0
1,1,3518,0
2,1,10967,1
3,1,10833,0
4,1,6450,0
...,...,...,...
4433,40,10746,1
4434,40,13111,0
4435,40,10306,0
4436,40,1196,0


## Trainning Data

In [22]:
RANK_CUTOFF = 100
SEED=42

from sklearn.model_selection import train_test_split

tr_va_topics, test_topics = train_test_split(query, test_size=9, random_state=SEED)
train_topics, valid_topics = train_test_split(tr_va_topics, test_size=3, random_state=SEED)


In [23]:
sdm = pt.rewrite.SDM()
qe = pt.rewrite.Bo1QueryExpansion(index)

In [50]:
qrels = qrels.rename(columns = {'relevance':'label'})


In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=SEED, n_jobs=2)

rf_pipe2 = ltr_feats_0 >> pt.ltr.apply_learned_model(rf)

%time rf_pipe2.fit(train_topics, qrels)

23:56:06.986 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
23:56:12.428 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
23:56:17.667 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
23:56:22.084 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    1.0s


CPU times: user 1min 18s, sys: 2.68 s, total: 1min 21s
Wall time: 1min 16s


[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    2.0s finished


In [ ]:
pip install fastrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 891 kB 8.5 MB/s 


In [ ]:
import fastrank
train_request = fastrank.TrainRequest.coordinate_ascent()

params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe = ltr_feats_0 >> pt.ltr.apply_learned_model(train_request, form='fastrank')
ca_pipe.fit(train_topics, qrels)

00:18:10.834 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:18:15.259 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:18:20.942 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:18:25.289 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

In [ ]:
import lightgbm as lgb

lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe = ltr_feats_0 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[20]})
lmart_x_pipe.fit(train_topics, qrels, valid_topics, qrels)

00:19:26.751 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:19:31.364 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:19:35.796 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:19:40.533 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [ ]:
pt.Experiment(
    [bm25 % RANK_CUTOFF,tfidf % RANK_CUTOFF, DFIZ % RANK_CUTOFF, DirichletLM % RANK_CUTOFF, rf_pipe2, ca_pipe, lmart_x_pipe], 
    valid_topics,#test_topics,
    qrels,
    names=["BM25", "TFIDF", "DFIZ", "DirichletLM", "BM25 + RF", "BM25 + CA", "BM25 + LMart"],
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"]
)

00:20:22.372 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:20:22.884 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:20:23.449 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:20:23.997 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    0.1s finished


00:20:30.189 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:20:30.744 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:20:31.331 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
00:20:31.907 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

,name,map,ndcg,ndcg_cut_10,mrt
0,BM25,0.403635,0.592480,0.372583,1264.922333
1,TFIDF,0.367917,0.540081,0.330209,1236.006960
2,DFIZ,0.302118,0.464726,0.219179,1240.347605
3,DirichletLM,0.351186,0.489888,0.244719,1222.668702
4,BM25 + RF,0.346819,0.560605,0.290438,2579.335419
5,BM25 + CA,0.457765,0.684742,0.559032,2554.309497
6,BM25 + LMart,0.332818,0.554152,0.264273,2536.095556


We fetched more data from imdb including tags, release_date, languages, runtime as so on.

In [ ]:
more_data = pd.read_csv("/content/drive/MyDrive/650data/merged_final_data.csv")
more_data.columns

Index(['Unnamed: 0', 'Unnamed: 0_x', 'imdbid', 'movie_title', 'release_date',
       'languages', 'countries_of_origin', 'production_companies', 'runtime',
       'plot_words', 'taglines', 'directors', 'writers', 'cast', 'parentguide',
       'Unnamed: 0_y', 'title', 'review_title_1', 'review_content_1',
       'review_title_2', 'review_content_2', 'review_title_3',
       'review_content_3', 'review_title_4', 'review_content_4',
       'review_title_5', 'review_content_5'],
      dtype='object')

In [42]:
ltr_feats_1 = (bm25 % RANK_CUTOFF) >> pt.text.get_text(index, ['review_title_1', 'review_content_1',
       'review_title_2', 'review_content_2', 'review_title_3',
       'review_content_3', 'review_title_4', 'review_content_4',
       'review_title_5', 'review_content_5']) >> (
    pt.transformer.IdentityTransformer()
    ** # sequential dependence
    (sdm >> bm25)
    ** # score of reviews
    (pt.text.scorer(body_attr="review_content_1", wmodel='BM25') + 
     pt.text.scorer(body_attr="review_content_2", wmodel='BM25') +
     pt.text.scorer(body_attr="review_content_3", wmodel='BM25') +
     pt.text.scorer(body_attr="review_content_4", wmodel='BM25') +
     pt.text.scorer(body_attr="review_content_5", wmodel='BM25') )
    **
    # (pt.apply.doc_score(lambda row: int( row["doi"] is not None and len(row["doi"]) > 0) ))
    pt.apply.doc_score(lambda row: int(len(row['review_content_1'])>300))
    **
    pt.apply.doc_score(lambda row: int(len(row['review_content_2'])>300))
    **
    pt.apply.doc_score(lambda row: int(len(row['review_content_3'])>300))
    **
    pt.apply.doc_score(lambda row: int(len(row['review_content_4'])>300))
    **
    pt.apply.doc_score(lambda row: int(len(row['review_content_5'])>300))
    ** # abstract coordinate match
    pt.BatchRetrieve(index, wmodel="CoordinateMatch")
)
                                                             
# for reference, lets record the feature names here too
fnames=["BM25", "SDM", "reviews", "review_length_1","review_length_2", "review_length_3", "review_length_4", "review_length_5", "CoordinateMatch"]

In [43]:
pt.Experiment(
    [ltr_feats_1 >> pt.ltr.feature_to_score(i) for i in range(len(fnames))],
    test_topics,
    qrels, 
    names=fnames,
    eval_metrics=["map", "ndcg", "ndcg_cut_100", "P_30", "recall_30", "mrt"])

02:59:09.324 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
02:59:11.311 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
02:59:13.329 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
02:59:15.386 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

,name,map,ndcg,ndcg_cut_100,P_30,recall_30,mrt
0,BM25,0.312432,0.525669,0.525669,0.362963,0.380114,3173.834589
1,SDM,0.314997,0.526676,0.526676,0.366667,0.383976,3166.775013
2,reviews,0.409315,0.615318,0.615318,0.422222,0.450673,2947.280054
3,review_length_1,0.227318,0.454826,0.454826,0.229630,0.231986,3038.732355
4,review_length_2,0.220436,0.449740,0.449740,0.222222,0.225511,3107.625412
5,review_length_3,0.222417,0.449098,0.449098,0.233333,0.235243,2881.075521
6,review_length_4,0.225408,0.453736,0.453736,0.229630,0.230250,2845.134774
7,review_length_5CoordinateMatch,0.224068,0.453299,0.453299,0.222222,0.226370,3219.827198


In [44]:
ltr_feats_2 = (bm25 % RANK_CUTOFF) >> pt.text.get_text(index, ['review_title_1', 'review_content_1',
       'review_title_2', 'review_content_2', 'review_title_3',
       'review_content_3', 'review_title_4', 'review_content_4',
       'review_title_5', 'review_content_5']) >> (
    pt.transformer.IdentityTransformer()
    ** # sequential dependence
    (sdm >> bm25)
    ** # score of reviews
    (pt.text.scorer(body_attr="review_content_1", wmodel='BM25') + 
     pt.text.scorer(body_attr="review_content_2", wmodel='BM25') +
     pt.text.scorer(body_attr="review_content_3", wmodel='BM25') +
     pt.text.scorer(body_attr="review_content_4", wmodel='BM25') +
     pt.text.scorer(body_attr="review_content_5", wmodel='BM25') )
    **
    # (pt.apply.doc_score(lambda row: int( row["doi"] is not None and len(row["doi"]) > 0) ))
    pt.apply.doc_score(lambda row: int(len(row['review_content_1'].split(" "))>300))
    **
    pt.apply.doc_score(lambda row: int(len(row['review_content_2'].split(" "))>300))
    **
    pt.apply.doc_score(lambda row: int(len(row['review_content_3'].split(" "))>300))
    **
    pt.apply.doc_score(lambda row: int(len(row['review_content_4'].split(" "))>300))
    **
    pt.apply.doc_score(lambda row: int(len(row['review_content_5'].split(" "))>300))
    ** # abstract coordinate match
    pt.BatchRetrieve(index, wmodel="CoordinateMatch")
)
                                                             
# for reference, lets record the feature names here too
fnames=["BM25", "SDM", "reviews", "review_length_1","review_length_2", "review_length_3", "review_length_4", "review_length_5", "CoordinateMatch"]

In [45]:
pt.Experiment(
    [ltr_feats_2 >> pt.ltr.feature_to_score(i) for i in range(len(fnames))],
    test_topics,
    qrels, 
    names=fnames,
    eval_metrics=["map", "ndcg", "ndcg_cut_100", "P_30", "recall_30", "mrt"])

03:04:27.219 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:04:29.262 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:04:31.446 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:04:33.432 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

,name,map,ndcg,ndcg_cut_100,P_30,recall_30,mrt
0,BM25,0.312432,0.525669,0.525669,0.362963,0.380114,2936.370252
1,SDM,0.314997,0.526676,0.526676,0.366667,0.383976,2839.809024
2,reviews,0.409315,0.615318,0.615318,0.422222,0.450673,2918.628418
3,review_length_1,0.241463,0.462844,0.462844,0.270370,0.259586,2986.118568
4,review_length_2,0.218752,0.458297,0.458297,0.225926,0.221056,2887.553854
5,review_length_3,0.215383,0.442862,0.442862,0.248148,0.250913,3207.373949
6,review_length_4,0.223439,0.463245,0.463245,0.222222,0.219294,3128.325957
7,review_length_5,0.231276,0.472418,0.472418,0.244444,0.237945,2955.162549
8,CoordinateMatch,0.287904,0.528188,0.528188,0.300000,0.305716,2916.165276


In [51]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=400, verbose=1, random_state=SEED, n_jobs=2)

rf_pipe2 = ltr_feats_2 >> pt.ltr.apply_learned_model(rf)

%time rf_pipe2.fit(train_topics, qrels)

03:24:20.857 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:24:27.066 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:24:33.135 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:24:39.164 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.4s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    1.4s


CPU times: user 2min 59s, sys: 1.07 s, total: 3min
Wall time: 1min 47s


[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    2.8s finished


In [48]:
pip install fastrank

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 891 kB 5.3 MB/s 


In [52]:
import fastrank
train_request = fastrank.TrainRequest.coordinate_ascent()

params = train_request.params
params.init_random = True
params.normalize = True
params.seed = 1234567

ca_pipe2 = ltr_feats_2 >> pt.ltr.apply_learned_model(train_request, form='fastrank')
ca_pipe2.fit(train_topics, qrels)

03:26:23.840 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:26:35.003 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:26:41.543 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:26:48.309 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

In [53]:
import lightgbm as lgb

lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

lmart_x_pipe2 = ltr_feats_2 >> pt.ltr.apply_learned_model(lmart_l, form="ltr", fit_kwargs={'eval_at':[20]})
lmart_x_pipe2.fit(train_topics, qrels, valid_topics, qrels)

03:28:29.791 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:28:36.643 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:28:45.670 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:28:52.131 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:118: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/usr/local/lib/python3.8/dist-packages/lightgbm/engine.py:123: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[1]	valid_0's ndcg@20: 0.350087
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's ndcg@20: 0.402333
[3]	valid_0's ndcg@20: 0.45096
[4]	valid_0's ndcg@20: 0.421204
[5]	valid_0's ndcg@20: 0.416481
[6]	valid_0's ndcg@20: 0.418987
[7]	valid_0's ndcg@20: 0.394481
[8]	valid_0's ndcg@20: 0.39214
Early stopping, best iteration is:
[3]	valid_0's ndcg@20: 0.45096


In [54]:
pt.Experiment(
    [bm25 % RANK_CUTOFF,tfidf % RANK_CUTOFF, DFIZ % RANK_CUTOFF, DirichletLM % RANK_CUTOFF, rf_pipe2, ca_pipe2, lmart_x_pipe2], 
    valid_topics,#test_topics,
    qrels,
    names=["BM25", "TFIDF", "DFIZ", "DirichletLM", "BM25 + RF", "BM25 + CA", "BM25 + LMart"],
    eval_metrics=["map", "ndcg", "ndcg_cut_10", "mrt"]
)

03:30:01.016 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:30:02.084 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:30:03.079 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:30:03.885 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 196 tasks      | elapsed:    0.0s
[Parallel(n_jobs=2)]: Done 400 out of 400 | elapsed:    0.1s finished


03:30:14.108 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:30:15.249 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:30:16.430 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
03:30:17.340 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

,name,map,ndcg,ndcg_cut_10,mrt
0,BM25,0.403635,0.592480,0.372583,2641.653243
1,TFIDF,0.367917,0.540081,0.330209,2036.653642
2,DFIZ,0.302118,0.464726,0.219179,2023.933711
3,DirichletLM,0.351186,0.489888,0.244719,2022.773561
4,BM25 + RF,0.360919,0.582864,0.309805,4386.255173
5,BM25 + CA,0.471322,0.680748,0.564821,4349.144405
6,BM25 + LMart,0.384094,0.632396,0.377155,4343.560337


In [85]:
def search_movie(input_query):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
    searched_docno = ca_pipe2.search(input_query)['docno'][:10].to_list()
    searched_title = final_document[final_document["docno"].isin(searched_docno)]['title']
    searched_imdbid = final_document[final_document["docno"].isin(searched_docno)]['imdb_id'].to_list()
    i = 0
    for title in searched_title: 
        i+=1
        print(str(i), title)
    return searched_imdbid

In [88]:
search_movie("Films depicting the modern history of China")

04:03:28.846 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
04:03:29.279 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
04:03:29.693 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Defaulting to global termpipelines configuration of 'Stopwords,PorterStemmer'. Set a termpipelines control to remove this warning.
04:03:30.869 [main] WARN org.terrier.querying.ApplyTermPipeline - The index has no termpipelines configuration, and no control configuration is found. Def

In [7]:
final_data = pd.read_csv("merged_final_data.csv").iloc[:,1:]
final_data = final_data.rename(columns = {'imdbid':'imdb_id'}).dropna()
final_data['docno'] = final_data.index + 1
final_data['docno'] = final_data.docno.astype(str)
final_data = final_data[['imdb_id', 'movie_title', 'release_date', 'languages',
       'countries_of_origin', 'production_companies', 'runtime', 'plot_words',
       'taglines', 'directors', 'writers', 'cast', 'parentguide']]

In [ ]:
import re
query = input("Type in a query you want to search")
imdbid_list = search_movie(query)
answer_1 = input("Do you want to know more of the movies? Type 1 for Yes and 2 for No.")
if int(answer_1) == 2:
    pass
else:
    movie_order = input("Type in the number of the movie you want to know about.")
    movie_imdbid = imdbid_list[int(movie_order)-1]
    row = final_data[final_data["imdb_id"] == movie_imdbid]
    metadata = list(row.to_dict(orient='index').values())[0]
    row = final_data[final_data["movie_title"] == "The Last Emperor"]
    metadata = list(row.to_dict(orient='index').values())[0]
    for k,v in metadata.items():
        print("\033[91m\033[1m" + " ".join([x.capitalize() for x in k.split("_")]) + "\033[0m\033[0m: ") 
        if k != "parentguide":
            print(" ",v) 
        else:
            for v_1 in re.split("., *",v):
                print(" ",v_1+".") 